In [1]:
from langchain_community.document_loaders import CSVLoader

loader = CSVLoader(
    file_path=r"C:\Users\bhushan\Desktop\dev1\semantic_st25\nco_data_with_desc_500_new.csv",
    content_columns=["description"],
    metadata_columns=["occupation_title", "nco_2015", "nco_2004", "division", "subdivision", "group", "family", "sno"]
)

documents = loader.load()

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [4]:
from langchain_community.vectorstores.faiss import FAISS

texts = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]

faiss_store = FAISS.from_texts(texts, embeddings, metadatas=metadatas)

In [17]:
import pandas as pd

query = "english teacher"
rows = []

results = faiss_store.similarity_search_with_score(query, k=50)

for doc, score in results:
    row = doc.metadata.copy()
    row["similarity_score"] = score
    rows.append(row)

df = pd.DataFrame(rows)
print(df[[
    "sno", "similarity_score", "occupation_title", "nco_2015", "nco_2004",
    "division"
]].to_string(index=False))

sno  similarity_score                                                   occupation_title  nco_2015 nco_2004      division
164          1.234366                                                         Headmaster 1345.0100  1229.20      Managers
466          1.353673                                                            Dresser 2240.0400  3221.40 Professionals
 40          1.361574                                                 Principal, College 1120.3900  1219.50      Managers
491          1.368798                                             Occupational Therapist 2264.0200  3226.20 Professionals
 95          1.386244                                                Manager, Plantation 1311.0800  1221.20      Managers
464          1.396817                                                   Physician, Unani 2230.0400  2224.10 Professionals
266          1.402133                                          Printing Master (Textile) 2141.1700  2149.50 Professionals
431          1.413092   